<a href="https://colab.research.google.com/github/ga642381/ML2021-Spring/blob/main/HW02/HW02-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **作业 2-1 音素分类**

* 幻灯片: https://speech.ee.ntu.edu.tw/~hylee/ml/ml2021-course-data/hw/HW02/HW02.pdf
* 视频 (中文): https://youtu.be/PdjXnQbu2zo
* 视频 (英文): https://youtu.be/ESRr-VCykBs


## DARPA TIMIT 声学-音素连续语音语料库 (TIMIT)
TIMIT阅读语音语料库的设计目的是提供语音数据，用于获取声学音素知识以及开发和评估自动语音识别系统。

本作业是一个多类别分类任务，
我们将训练一个深度神经网络分类器，从TIMIT语音语料库中预测每个帧的音素。

链接: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## 下载数据
从谷歌云盘下载数据，然后解压。

运行此代码块后，您应该有 `timit_11/train_11.npy`、`timit_11/train_label_11.npy` 和 `timit_11/test_11.npy`。<br><br>
`timit_11/`
- `train_11.npy`: 训练数据<br>
- `train_label_11.npy`: 训练标签<br>
- `test_11.npy`: 测试数据<br><br>

**注意：如果谷歌云盘链接失效，您可以直接从Kaggle下载数据并上传到工作区**




In [1]:
# !gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
# !unzip data.zip
# !ls 

## 准备数据
从`.npy`文件（NumPy数组）中加载训练和测试数据。

In [2]:
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler

print('正在加载数据...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('原始训练数据大小: {}'.format(train.shape))
print('原始测试数据大小: {}'.format(test.shape))
USE_FEATURE_SELECTION=True
USE_NORMALIZATION=True
N_FEATURES=200
# 特征选择 - 使用ANOVA F-值找出最重要的特征
if USE_FEATURE_SELECTION:
    print(f'正在选择最重要的 {N_FEATURES} 个特征...')
    selector = SelectKBest(f_classif, k=N_FEATURES)
    # 使用所有训练数据进行特征选择
    train_new = selector.fit_transform(train, train_label)
    test_new = selector.transform(test)
    # 打印特征选择的分数
    import pandas as pd
    
    # 获取特征分数
    feature_scores = selector.scores_
    
    # 创建特征索引的DataFrame
    df_columns = pd.DataFrame([f"特征_{i}" for i in range(train.shape[1])])
    df_scores = pd.DataFrame(feature_scores)
    
    # 合并两个DataFrame以便更好地可视化
    feature_scores_df = pd.concat([df_columns, df_scores], axis=1)
    feature_scores_df.columns = ['特征名称', '分数']
    
    # 打印分数最高的20个特征
    print("分数最高的20个特征:")
    print(feature_scores_df.nlargest(200, '分数'))
    # 获取被选中的特征索引
    selected_indices = selector.get_support(indices=True)
    print(f'选定的特征索引: {selected_indices[:10]}... (共{len(selected_indices)}个)')
    
    # 更新训练和测试数据
    train = train_new
    test = test_new
    print(f'特征选择后训练数据大小: {train.shape}')
    print(f'特征选择后测试数据大小: {test.shape}')

# 归一化处理
if USE_NORMALIZATION:
    print('正在进行数据归一化...')
    scaler = StandardScaler()
    train = scaler.fit_transform(train)
    test = scaler.transform(test)
    print('归一化完成')

正在加载数据...
原始训练数据大小: (1229932, 429)
原始测试数据大小: (451552, 429)
正在选择最重要的 200 个特征...
分数最高的20个特征:
       特征名称            分数
235  特征_235  83663.352355
274  特征_274  71385.954938
196  特征_196  59692.624024
313  特征_313  46536.203068
157  特征_157  36152.184568
..      ...           ...
208  特征_208   3047.407233
305  特征_305   2996.665994
139  特征_139   2975.303450
61    特征_61   2946.420719
266  特征_266   2927.731564

[200 rows x 2 columns]
选定的特征索引: [ 0  1  2  3  4 13 14 15 16 17]... (共200个)
特征选择后训练数据大小: (1229932, 200)
特征选择后测试数据大小: (451552, 200)
正在进行数据归一化...
归一化完成


In [3]:
# feature_scores_df.nlargest(200, '分数')

In [24]:
# ==================== 所有配置参数 ====================
# 数据相关配置
VAL_RATIO = 0.2              # 验证集比例（仅在非交叉验证时使用）
BATCH_SIZE = 256             # 批处理大小
N_FOLDS = 3                  # 交叉验证折数
USE_CV = False               # 是否使用交叉验证
 
# 随机种子配置
SEED = 182142                # 随机种子值

# 特征选择与预处理配置
N_FEATURES = 200             # 选择的特征数量
USE_FEATURE_SELECTION = True # 是否进行特征选择
USE_NORMALIZATION = True     # 是否进行归一化

# 训练相关配置
NUM_EPOCH = 200              # 训练轮数
LEARNING_RATE = 0.0001       # 学习率
WEIGHT_DECAY = 1e-4          # L2正则化系数
MODEL_PATH = './model.ckpt'  # 模型保存路径

# 学习率调度器配置
USE_SCHEDULER = True         # 是否使用学习率调度器
SCHEDULER_PATIENCE = 5       # 学习率调度器的耐心值 (几轮不提升就降低)
SCHEDULER_FACTOR = 0.1       # 学习率缩放因子 (new_lr = old_lr * factor)

# ==================== 应用配置 ====================


## 创建数据集

In [4]:
import torch
from torch.utils.data import Dataset

# 这段代码定义了一个名为TIMITDataset的自定义数据集类，继承自PyTorch的Dataset类
class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        # 初始化方法接收特征数据X和可选的标签数据y
        # 将NumPy数组X转换为PyTorch浮点张量
        self.data = torch.from_numpy(X).float()
        if y is not None:
            # 如果提供了标签数据y
            # 将y转换为整数类型
            y = y.astype(int)
            # 然后转换为PyTorch长整型张量
            self.label = torch.LongTensor(y)
        else:
            # 如果没有提供标签数据，则标签设为None（用于测试集）
            self.label = None

    def __getitem__(self, idx):
        # 获取指定索引的数据项
        if self.label is not None:
            # 如果有标签，返回(数据,标签)对
            return self.data[idx], self.label[idx]
        else:
            # 如果没有标签，只返回数据
            return self.data[idx]

    def __len__(self):
        # 返回数据集的大小（样本数量）
        return len(self.data)


将标记数据分为训练集和验证集，您可以修改变量 `VAL_RATIO` 来更改验证数据的比例。

In [10]:
# 这段代码的作用是将原始的训练数据划分为训练集和验证集。
# 使用统一配置文件中定义的VAL_RATIO参数

# 计算训练集的样本数量
# train.shape[0] 表示训练数据train的样本数量（即行数）
# 这里的 shape[0] 表示样本数量（行数），shape[1] 表示特征数量（列数）
# 如果写成 train.shape[1]，得到的是特征的数量，而不是样本数量
# 例如，如果 train.shape 是 (1000, 39)，那么 train.shape[0]=1000，train.shape[1]=39
# 这里用于切分数据集的应该是样本数量，所以用 shape[0]
# 你也可以尝试填入其他数值，比如直接写一个整数（如 800），那就是前800个样本作为训练集

percent = int(train.shape[0] * (1 - VAL_RATIO))

# 根据计算得到的数量，将数据切分为训练集和验证集
# train[:percent] 和 train_label[:percent] 分别是训练集的特征和标签
# train[percent:] 和 train_label[percent:] 分别是验证集的特征和标签
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]

# 打印训练集和验证集的大小，方便检查切分是否正确
print('训练集大小: {}'.format(train_x.shape))
print('验证集大小: {}'.format(val_x.shape))

训练集大小: (983945, 200)
验证集大小: (245987, 200)


从数据集创建数据加载器，您可以随意调整这里的变量 `BATCH_SIZE`。

In [11]:
from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #只打乱训练数据
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

清理不需要的变量以节省内存。<br>

**注意：如果您稍后需要使用这些变量，则可以删除此代码块或稍后清理不需要的变量<br>数据大小相当大，所以请注意colab中的内存使用情况**

In [ ]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

63

## 创建模型

定义模型架构，鼓励您更改和试验模型架构。

In [13]:
# 这段代码定义了一个用于分类任务的神经网络模型，名为Classifier，继承自PyTorch的nn.Module。
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self, input_dim):
        super(Classifier, self).__init__()
        # 定义神经网络结构
        self.net = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.LeakyReLU(),
            nn.BatchNorm1d(1024),  # 修正BN层的维度
            nn.Dropout(p=0.2),  # 添加Dropout减少过拟合
            
            nn.Linear(1024, 512),
            nn.LeakyReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(p=0.2),
            
            nn.Linear(512, 128),
            nn.LeakyReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(p=0.2),
            
            nn.Linear(128, 39)  # 输出39个类别
        )
        # 定义损失函数为交叉熵损失，适用于分类任务
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        # 前向传播
        return self.net(x)

    def cal_loss(self, pred, target):
        # 计算损失
        return self.criterion(pred, target)


## 训练

In [14]:
#检查设备
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

固定随机种子以确保结果可重现。

In [15]:
# 固定随机种子
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [ ]:
# help(torch.optim.Adam)

您可以随意更改这里的训练参数。

In [26]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

# 固定随机种子以确保结果可重现
same_seeds(SEED)

# 获取设备
device = get_device()
print(f'设备: {device}')

# 根据错误信息，Classifier类需要input_dim参数
# 确保在创建模型实例时提供input_dim参数
# 例如：model = Classifier(input_dim=YOUR_INPUT_DIM).to(device)

# 创建模型，定义损失函数和优化器
# 注意：此处的 train_x.shape[1] 必须在 USE_CV=False 时才有效
# 因为在交叉验证模式下，train_x 可能尚未定义
if not USE_CV:
    model = Classifier(input_dim=train_x.shape[1]).to(device)
else:
    # 在CV模式下，我们将在循环内创建模型，此处先创建一个占位符
    model = Classifier(input_dim=N_FEATURES).to(device)

criterion = nn.CrossEntropyLoss() 
# 添加L2正则化（weight_decay参数）
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# 初始化学习率调度器
scheduler = None
if USE_SCHEDULER and not USE_CV:
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=SCHEDULER_FACTOR, patience=SCHEDULER_PATIENCE)
    print("已为常规训练启用学习率调度器。")


设备: cuda
已为常规训练启用学习率调度器。


In [18]:
import torch

# 检查是否有CUDA可用
print(f"CUDA是否可用: {torch.cuda.is_available()}")

# 查看可用的CUDA设备数量
print(f"CUDA设备数量: {torch.cuda.device_count()}")

# 查看当前CUDA设备
print(f"当前CUDA设备索引: {torch.cuda.current_device()}")

# 获取当前设备名称
print(f"当前CUDA设备名称: {torch.cuda.get_device_name(torch.cuda.current_device())}")

# 获取设备属性
if torch.cuda.is_available():
    device_props = torch.cuda.get_device_properties(torch.cuda.current_device())
    print(f"设备总内存: {device_props.total_memory / 1024**3:.2f} GB")
    print(f"多处理器数量: {device_props.multi_processor_count}")
    print(f"CUDA能力: {device_props.major}.{device_props.minor}")




CUDA是否可用: True
CUDA设备数量: 1
当前CUDA设备索引: 0
当前CUDA设备名称: NVIDIA GeForce RTX 4070 Laptop GPU
设备总内存: 8.00 GB
多处理器数量: 36
CUDA能力: 8.9


In [19]:
# 在代码中执行系统命令
import os
os.system("nvidia-smi")

0

In [21]:
# 进行3折交叉验证
from sklearn.model_selection import KFold
import copy
import time

# 只有在启用交叉验证时才执行
if USE_CV:
    print("开始进行3折交叉验证...")
    
    # 重新加载全部训练数据（若之前已删除）
    data_root='./timit_11/'
    if 'train' not in locals() or 'train_label' not in locals():
        print("重新加载训练数据...")
        train = np.load(data_root + 'train_11.npy')
        train_label = np.load(data_root + 'train_label_11.npy')
    
    # 应用特征选择和归一化
    if USE_FEATURE_SELECTION and 'selector' not in locals():
        print(f'正在选择最重要的 {N_FEATURES} 个特征...')
        selector = SelectKBest(f_classif, k=N_FEATURES)
        train = selector.fit_transform(train, train_label)
        test = selector.transform(test)
        print(f'特征选择后数据大小: {train.shape}')
        
    if USE_NORMALIZATION and 'scaler' not in locals():
        print('正在进行数据归一化...')
        scaler = StandardScaler()
        train = scaler.fit_transform(train)
        test = scaler.transform(test)
        print('归一化完成')
    
    # 初始化K折交叉验证
    kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
    fold_val_accs = []
    
    # 早停参数
    PATIENCE = 5  # 连续10轮验证集性能不提升则停止
    
    # 开始K折循环
    for fold, (train_idx, val_idx) in enumerate(kf.split(train)):
        print(f"\n=== 第 {fold+1}/{N_FOLDS} 折训练开始 ===")
        
        # 准备当前折的数据
        train_x, train_y = train[train_idx], train_label[train_idx]
        val_x, val_y = train[val_idx], train_label[val_idx]
        
        # 创建数据加载器
        train_set = TIMITDataset(train_x, train_y)
        val_set = TIMITDataset(val_x, val_y)
        train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)
        
        # 创建新模型实例
        fold_model = Classifier(input_dim=train_x.shape[1]).to(device)
        fold_criterion = nn.CrossEntropyLoss()
        fold_optimizer = torch.optim.Adam(fold_model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
        
        # 训练当前折
        best_val_acc = 0.0
        fold_path = f'./model_fold_{fold+1}.ckpt'
        
        # 早停计数器
        patience_counter = 0
        
        for epoch in range(NUM_EPOCH):
            # 训练阶段
            fold_model.train()
            train_loss = 0.0
            train_acc = 0.0
            
            for i, data in enumerate(train_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                
                fold_optimizer.zero_grad()
                outputs = fold_model(inputs)
                batch_loss = fold_criterion(outputs, labels)
                batch_loss.backward()
                fold_optimizer.step()
                
                _, train_pred = torch.max(outputs, 1)
                train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
                train_loss += batch_loss.item()
            
            # 验证阶段
            fold_model.eval()
            val_loss = 0.0
            val_acc = 0.0
            
            with torch.no_grad():
                for i, data in enumerate(val_loader):
                    inputs, labels = data
                    inputs, labels = inputs.to(device), labels.to(device)
                    
                    outputs = fold_model(inputs)
                    batch_loss = fold_criterion(outputs, labels)
                    
                    _, val_pred = torch.max(outputs, 1)
                    val_acc += (val_pred.cpu() == labels.cpu()).sum().item()
                    val_loss += batch_loss.item()
            
            # 计算准确率
            train_acc /= len(train_set)
            val_acc /= len(val_set)
            train_loss /= len(train_loader)
            val_loss /= len(val_loader)
            
            # 输出当前训练状态
            print(f'[折 {fold+1}, 轮 {epoch+1}/{NUM_EPOCH}] 训练准确率: {train_acc:.6f} 损失: {train_loss:.6f} | 验证准确率: {val_acc:.6f} 损失: {val_loss:.6f}')
            
            # 保存最佳模型
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                torch.save(fold_model.state_dict(), fold_path)
                print(f'保存第 {fold+1} 折最佳模型，准确率: {best_val_acc:.6f}')
                patience_counter = 0  # 重置早停计数器
            else:
                patience_counter += 1  # 增加早停计数器
                
            # 早停检查
            if patience_counter >= PATIENCE:
                print(f'验证准确率连续 {PATIENCE} 轮未提升，提前停止训练')
                break
        
        # 记录当前折的最佳验证准确率
        fold_val_accs.append(best_val_acc)
        print(f"第 {fold+1} 折最佳验证准确率: {best_val_acc:.6f}")
    
    # 输出交叉验证结果
    cv_avg = sum(fold_val_accs) / len(fold_val_accs)
    print(f"\n3折交叉验证平均准确率: {cv_avg:.6f}")
    print(f"各折验证准确率: {fold_val_accs}")
    
    # 在全部训练数据上训练最终模型
    print("\n在全部训练数据上训练最终模型...")
    
    # 准备全部训练数据
    full_train_set = TIMITDataset(train, train_label)
    full_train_loader = DataLoader(full_train_set, batch_size=BATCH_SIZE, shuffle=True)
    
    # 创建最终模型
    final_model = Classifier().to(device)
    final_criterion = nn.CrossEntropyLoss()
    final_optimizer = torch.optim.Adam(final_model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    
    # 训练最终模型（使用早停）
    best_acc = 0.0
    patience_counter = 0
    best_model_state = None
    
    for epoch in range(NUM_EPOCH):
        final_model.train()
        train_loss = 0.0
        train_acc = 0.0
        
        for i, data in enumerate(full_train_loader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            
            final_optimizer.zero_grad()
            outputs = final_model(inputs)
            batch_loss = final_criterion(outputs, labels)
            batch_loss.backward()
            final_optimizer.step()
            
            _, train_pred = torch.max(outputs, 1)
            train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
            train_loss += batch_loss.item()
        
        # 计算训练准确率
        train_acc /= len(full_train_set)
        train_loss /= len(full_train_loader)
        
        # 输出训练状态
        print(f'[最终模型 轮 {epoch+1}/{NUM_EPOCH}] 训练准确率: {train_acc:.6f} 损失: {train_loss:.6f}')
        
        # 保存最佳模型（基于训练准确率，因为没有验证集）
        if train_acc > best_acc:
            best_acc = train_acc
            best_model_state = copy.deepcopy(final_model.state_dict())
            print(f'保存最佳模型，准确率: {best_acc:.6f}')
            patience_counter = 0
        else:
            patience_counter += 1
            
        # 早停检查
        if patience_counter >= PATIENCE:
            print(f'训练准确率连续 {PATIENCE} 轮未提升，提前停止训练')
            break
    
    # 加载最佳模型状态
    final_model.load_state_dict(best_model_state)
    
    # 保存最终模型
    torch.save(final_model.state_dict(), MODEL_PATH)
    print(f"最终模型已保存到 {MODEL_PATH}")
    
    # 使用最终模型进行预测
    model = final_model
else:
    print("跳过交叉验证，使用常规训练...")
    # 这里会执行原来的训练代码

开始进行3折交叉验证...

=== 第 1/3 折训练开始 ===
[折 1, 轮 1/50] 训练准确率: 0.566922 损失: 1.484077 | 验证准确率: 0.648306 损失: 1.128509
保存第 1 折最佳模型，准确率: 0.648306
[折 1, 轮 2/50] 训练准确率: 0.629127 损失: 1.195280 | 验证准确率: 0.674697 损失: 1.027675
保存第 1 折最佳模型，准确率: 0.674697
[折 1, 轮 3/50] 训练准确率: 0.647635 损失: 1.122423 | 验证准确率: 0.686378 损失: 0.979450
保存第 1 折最佳模型，准确率: 0.686378
[折 1, 轮 4/50] 训练准确率: 0.659736 损失: 1.075178 | 验证准确率: 0.696479 损失: 0.942951
保存第 1 折最佳模型，准确率: 0.696479
[折 1, 轮 5/50] 训练准确率: 0.669552 损失: 1.040021 | 验证准确率: 0.702262 损失: 0.919054
保存第 1 折最佳模型，准确率: 0.702262
[折 1, 轮 6/50] 训练准确率: 0.677645 损失: 1.011703 | 验证准确率: 0.708426 损失: 0.897639
保存第 1 折最佳模型，准确率: 0.708426
[折 1, 轮 7/50] 训练准确率: 0.683079 损失: 0.988557 | 验证准确率: 0.714233 损失: 0.878575
保存第 1 折最佳模型，准确率: 0.714233
[折 1, 轮 8/50] 训练准确率: 0.688657 损失: 0.969441 | 验证准确率: 0.717551 损失: 0.864509
保存第 1 折最佳模型，准确率: 0.717551
[折 1, 轮 9/50] 训练准确率: 0.693584 损失: 0.952226 | 验证准确率: 0.721961 损失: 0.850505
保存第 1 折最佳模型，准确率: 0.721961
[折 1, 轮 10/50] 训练准确率: 0.697372 损失: 0.937330 | 验证准确率: 0.724912 损失:

KeyboardInterrupt: 

In [23]:
# 开始训练

best_acc = 0.0
for epoch in range(NUM_EPOCH):
    # 初始化训练和验证的准确率和损失
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    
    # 早停相关变量，与交叉验证部分类似
    patience_counter = 0 if 'patience_counter' not in locals() else patience_counter

    # 训练
    model.train() # 将模型设置为训练模式
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs) 
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # 获取具有最高概率的类的索引
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # 验证
    if len(val_set) > 0:
        model.eval() # 将模型设置为评估模式
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # 获取具有最高概率的类的索引
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] 训练准确率: {:3.6f} 损失: {:3.6f} | 验证准确率: {:3.6f} 损失: {:3.6f}'.format(
                epoch + 1, NUM_EPOCH, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # 如果模型改进，在此轮保存检查点
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), MODEL_PATH)
                print('保存模型，准确率 {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] 训练准确率: {:3.6f} 损失: {:3.6f}'.format(
            epoch + 1, NUM_EPOCH, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# 如果不进行验证，保存最后一轮
if len(val_set) == 0:
    torch.save(model.state_dict(), MODEL_PATH)
    print('在最后一轮保存模型')


[001/200] 训练准确率: 0.568413 损失: 1.479155 | 验证准确率: 0.650388 损失: 1.125137
保存模型，准确率 0.650
[002/200] 训练准确率: 0.629778 损失: 1.191593 | 验证准确率: 0.674567 损失: 1.028039
保存模型，准确率 0.675
[003/200] 训练准确率: 0.648529 损失: 1.120247 | 验证准确率: 0.686180 损失: 0.983682
保存模型，准确率 0.686
[004/200] 训练准确率: 0.660057 损失: 1.073676 | 验证准确率: 0.694707 损失: 0.948256
保存模型，准确率 0.695
[005/200] 训练准确率: 0.669855 损失: 1.038205 | 验证准确率: 0.702532 损失: 0.921561
保存模型，准确率 0.703
[006/200] 训练准确率: 0.677280 损失: 1.010579 | 验证准确率: 0.708671 损失: 0.899760
保存模型，准确率 0.709
[007/200] 训练准确率: 0.684220 损失: 0.987131 | 验证准确率: 0.714267 损失: 0.879771
保存模型，准确率 0.714
[008/200] 训练准确率: 0.688609 损失: 0.969237 | 验证准确率: 0.717557 损失: 0.866280
保存模型，准确率 0.718
[009/200] 训练准确率: 0.693515 损失: 0.952244 | 验证准确率: 0.721540 损失: 0.853192
保存模型，准确率 0.722
[010/200] 训练准确率: 0.697713 损失: 0.935911 | 验证准确率: 0.724541 损失: 0.843046
保存模型，准确率 0.725
[011/200] 训练准确率: 0.701091 损失: 0.923791 | 验证准确率: 0.726675 损失: 0.832636
保存模型，准确率 0.727
[012/200] 训练准确率: 0.704990 损失: 0.911420 | 验证准确率: 0.730187 损失: 0.82

KeyboardInterrupt: 

## 测试

创建测试数据集，并从保存的检查点加载模型。

In [ ]:
# 创建测试数据集
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# 创建模型并从检查点加载权重
model = Classifier().to(device)
model.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

进行预测。

In [ ]:
predict = []
model.eval() # 将模型设置为评估模式
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # 获取具有最高概率的类的索引

        for y in test_pred.cpu().numpy():
            predict.append(y)

将预测结果写入CSV文件。

运行此代码块后，从左侧文件部分下载`prediction.csv`文件并提交到Kaggle。

In [ ]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))